# Associative Dynamics in a Chaotic Neural Network
## M. Adachi, K. Aihara
*Neural Networks **10**, 1, 8348 (1997)*

$$
x_i(t+1) = f[\eta_i(t+1)+\zeta_i(t+1)]
$$

$$
\eta_i(t+1)=\sum \limits_j^N w_{ij}x_j(t)+k_f \eta_i(t)
$$

$$
\zeta_i(t+1)=k_r \zeta_i(t) - \alpha x_i(t) + a_i
$$

- $k_f$ - decay of feedback
- $k_r$ - decay of refract.
- $\alpha$ - refractory scaling

Uczymy za pomocą reguły Hebba:
$$
w_{ij} = \frac{1}{P} \sum \limits_p^P x_i^{(p)} x_j^{(p)}
$$

Obserwujemy przekrycie (overlap) ze wzorcem $p$:
$$
\mu_p = \frac{1}{N} \sum \limits_i^N x_i^{(p)}(2x_i-1)
$$

In [ ]:
%pylab

In [ ]:
# fcja aktywacji
# f=lambda y,ϵ=0.015: 1/(1+exp(-y/ϵ));
f=lambda y,ϵ=0.03: (tanh(y/ϵ)+1)/2;

In [ ]:
L=10
N=L*L

# przykładowe wzorce
patterns=2*array([
    [0,0,0,0,0,1,0,0,0,0,
    0,0,0,0,1,1,1,0,0,0,
    0,0,0,0,1,1,1,0,0,0,
    0,0,0,1,1,1,1,1,0,0,
    0,0,0,1,1,0,1,1,0,0,
    0,0,1,1,1,0,1,1,1,0,
    0,0,1,1,0,0,0,1,1,0,
    0,1,1,1,0,0,0,1,1,1,
    0,1,1,1,1,1,1,1,1,1,
    0,1,1,1,1,1,1,1,1,1],

    [0,0,1,1,1,0,0,0,1,1,
    0,1,1,1,1,1,1,1,1,1,
    1,1,1,0,1,1,1,1,0,0,
    1,1,0,0,0,1,1,0,0,0,
    0,0,0,0,0,0,0,0,0,0,
    0,0,0,1,1,0,0,0,1,1,
    0,0,1,1,1,1,0,1,1,1,
    1,1,1,1,1,1,1,1,1,0,
    1,1,0,0,0,1,1,1,0,0,
    0,0,0,0,0,0,0,0,0,0],

    [0,0,1,0,0,0,0,1,0,0,
    0,0,1,1,0,0,1,1,0,0,
    0,0,1,1,1,1,1,1,0,0,
    0,0,1,1,1,1,1,1,0,0,
    0,0,1,1,1,1,1,1,0,0,
    0,1,1,1,1,1,1,1,1,0,
    1,1,1,1,1,1,1,1,1,1,
    0,0,0,1,1,1,1,0,0,0,
    0,0,0,0,1,1,0,0,0,0,
    0,0,0,0,0,1,0,0,0,0],

    [1,1,0,0,0,0,0,0,1,1,
    1,1,1,0,0,0,0,1,1,1,
    0,1,1,1,0,0,1,1,1,0,
    0,0,1,1,1,1,1,1,0,0,
    0,0,0,1,1,1,0,0,0,0,
    0,0,0,0,1,1,1,0,0,0,
    0,0,1,1,1,1,1,1,0,0,
    0,1,1,1,0,0,1,1,1,0,
    1,1,1,0,0,0,0,1,1,1,
    1,1,0,0,0,0,0,0,1,1]
])-1

# wzorzec nieznany
unknown = 2*array(
    [1,1,1,1,1,1,1,1,1,1,
    1,0,0,1,1,1,1,0,0,1,
    1,0,0,0,0,0,0,0,0,1,
    1,1,0,0,0,0,0,0,1,1,
    1,1,0,0,0,0,0,0,1,1,
    1,1,0,0,0,0,0,0,1,1,
    1,1,0,0,0,0,0,0,1,1,
    1,0,0,0,0,0,0,0,0,1,
    1,0,0,1,1,1,1,0,0,1,
    1,1,1,1,1,1,1,1,1,1]
)-1

In [ ]:
i=1
for p in [*patterns,unknown]:
    subplot(1,5,i)
    i=i+1
    imshow(p.reshape(L,L))

In [ ]:
# kf=0.2 # decay of feedback
# kr=0.9 # decay of refract.
# alpha=10 # refractory scaling

# parametry z pracy Hirata, Oku, Aihara, Chaos 22, 047511 (2012)
kf=0.8 # decay of feedback
kr=0.9 # decay of refract.
alpha=12 # refractory scaling

x=zeros(N)  # outputs
eta=zeros(N)  # feedback
zeta=zeros(N) # refractoriness
w=zeros((N,N));

In [ ]:
# reguła Hebba

def train():
    global w
    w=patterns.T @ patterns / patterns.shape[0]
    w-=eye(N,N)

train()

In [ ]:
def one_step():
    global x, w, zeta, eta, a
    zeta=kr*zeta-alpha*x+a
    eta=kf*eta+w@x
    x=f(eta+zeta)

In [ ]:
def overlap(x,pat):
   return (2*x-1) @ pat / N

In [ ]:
figsize(12,8)

steps=100
# steps=1000

x=rand(N)
overlaps=zeros((patterns.shape[0],steps))
a=6.4
# a=6.4 + patterns[1]

# transient
for t in range(100):
    one_step()

cols=10

for t in range(steps):
    one_step()

    subplot(steps//cols,cols,t+1); imshow(x.reshape(10,10))
    
    for i in range(patterns.shape[0]):
        overlaps[i,t]=overlap(x,patterns[i])

In [ ]:
i=1
for o in overlaps:
    subplot(2,2,i)
    plot(o,"k-")
    i=i+1

In [ ]:
i=1
for o in overlaps:
    subplot(2,2,i)
    plot(o[:-1],o[1:],"k.", alpha=0.3)
    i+=1

In [ ]:
for o in overlaps:
    print(sum(abs(o))/steps)